In [2]:
from nltk.stem.isri import ISRIStemmer
import json
import pickle
import nltk
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
import random



In [3]:
stem=ISRIStemmer()
words=[]
classes=[]
documents=[]
ignore_words=['؟','!','.']
data_file=open('intentsTwo.json').read()
intents=json.loads(data_file)



In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
print(words)
words=[stem.stem(w) for w in words if w not in ignore_words]

['اهلا', 'هاي', 'كيف', 'حالك', '؟', 'السلام', 'عليكم', 'اشوفك', 'لاحقا', 'مع', 'السلامة', 'بااي', 'انا', 'همشي', 'بقا', 'بتمنالك', 'يوم', 'سعيد']


In [6]:
words=sorted(list(set(words)))
classes=sorted(list(set(classes)))

In [7]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [8]:
training=[]
output_empty=[0]*len(classes)
for doc in documents:
    bag=[]
    pattern_words=doc[0]
    
    pattern_words=[stem.stem(word) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])


In [9]:
random.shuffle(training)
training=np.array(training)
train_X=list(training[:,0])
train_Y=list(training[:,1])

C:\Users\CRIZMA MEGA STORE\AppData\Local\Temp\ipykernel_8364\3988184503.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [10]:
train_X

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]]

In [11]:
train_Y

[[0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [0, 1]]

In [12]:
model=Sequential()
model.add(Dense(64,input_shape=(len(train_X[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]),activation='softmax'))

sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)

model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist=model.fit(np.array(train_X),np.array(train_Y),epochs=200,batch_size=5,verbose=1)

D:\anaconda\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
2/2 [==============================] - 3s 103ms/step - loss: 0.7991 - accuracy: 0.4444
Epoch 2/200
2/2 [==============================] - 0s 13ms/step - loss: 0.7581 - accuracy: 0.3333
Epoch 3/200
2/2 [==============================] - 0s 14ms/step - loss: 0.7563 - accuracy: 0.2222
Epoch 4/200
2/2 [==============================] - 0s 13ms/step - loss: 0.7841 - accuracy: 0.5556
Epoch 5/200
2/2 [==============================] - 0s 12ms/step - loss: 0.6803 - accuracy: 0.5556
Epoch 6/200
2/2 [==============================] - 0s 13ms/step - loss: 0.6737 - accuracy: 0.6667
Epoch 7/200
2/2 [==============================] - 0s 14ms/step - loss: 0.6165 - accuracy: 0.8889
Epoch 8/200
2/2 [==============================] - 0s 13ms/step - loss: 0.5971 - accuracy: 0.5556
Epoch 9/200
2/2 [==============================] - 0s 10ms/step - loss: 0.5167 - accuracy: 0.7778
Epoch 10/200
2/2 [==============================] - 0s 6ms/step - loss: 0.6658 - accuracy: 0.7778
Epoch 11/200
2/2 [=

2/2 [==============================] - 0s 11ms/step - loss: 0.2056 - accuracy: 0.8889
Epoch 167/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0082 - accuracy: 1.0000
Epoch 168/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0258 - accuracy: 1.0000
Epoch 169/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0252 - accuracy: 1.0000
Epoch 170/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0229 - accuracy: 1.0000
Epoch 171/200
2/2 [==============================] - 0s 12ms/step - loss: 0.1296 - accuracy: 0.8889
Epoch 172/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 173/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0704 - accuracy: 1.0000
Epoch 174/200
2/2 [==============================] - 0s 11ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 175/200
2/2 [==============================] - 0s 11ms/step - loss: 0.0772 - accuracy: 1.0000
Epoch 176/200


In [13]:
model.save('chatbot.h5',hist)
print('moel created')

moel created
